### Libaries

In [1]:
import robotic as ry
import numpy as np
import time
from scipy.interpolate import Rbf
from scipy.interpolate import CubicSpline
print('version:', ry.__version__, ry.compiled())

version: 0.1.3 compile time: Dec 15 2023 11:21:37


### Initalise configuration

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C.view()

0

### Define color map

In [3]:
color_map = [[1,0,.5], # pink
             [0.5,1,0], # lime green
             [.5,1,1], # light turqois
             [0.5,0,1], # violet
             [1,0,0], # red
             [0,1,0], # green
             [0,0,1], # blue
             [1,.5,0], # orange
             [1,1,.5], # lime yellow
             [1,.5,1], # light pink                            
]

### Gripper functions

In [4]:
def gripper_open(C, bot, widht = 0.75, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

def gripper_close(bot, widht = 0.01, speed = 0.2):
    bot.gripperMove(ry._left, widht, speed)

    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)

def gripper_close_grasp(bot, target_obj = '', force=5.00, widht = 0.01, speed = 0.2):
    bot.gripperCloseGrasp(ry._left, target_obj, force, widht, speed)
    while not bot.gripperDone(ry._left):
        bot.sync(C, .1)
    

### Create N number of building cuboids

In [5]:
def create_n_boxes(C, N, position='fixed'):

    for i in range(N):
        box_name = 'box{}'.format(i + 1)

        if position == 'fixed':
            position_val1 = 0.1 * (i - 1)
            C.addFrame(box_name) \
                .setPosition([position_val1, 0.35, 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)


        elif position == 'random':
            C.addFrame(box_name) \
                .setPosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), 0.70]) \
                .setShape(ry.ST.ssBox, size=[0.05, 0.05, 0.10, 0.005]) \
                .setColor(color_map[i % len(color_map)]) \
                .setContact(True) \
                .setMass(1e-2)
    
    return C

### Define construction site

In [6]:
def create_construction_site(C, position = 'fixed'):
    
    if position == 'fixed':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.275, .275, .090])
        target_location.setRelativePosition([-.5, -.225, .0])
        target_location.setColor([1., 0.75, 0.])
        return target_location
    
    elif position == 'random':
        target_location = C.addFrame('construction_site', 'table')
        target_location.setShape(ry.ST.box, size = [.3, .3, .1])
        target_location.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
        target_location.setColor([1, 0.75, 0])
        return target_location
        

In [7]:
def create_pillar_target_postion(C, position = 'fixed', n_pillars = 2):
    
    if position == 'fixed':
        for i in range(0, n_pillars):
            target_pillar = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .090])
            target_pillar.setRelativePosition([-.45 - (i / 10) , -.225, .0])
            target_pillar.setColor([0,1,0])
    
    elif position == 'random':
        for i in range(0, n_pillars):
            C = C.addFrame('pillar_target_pos' + str(i+1), 'table')
            target_pillar.setShape(ry.ST.box, size=[.065, .065, .090])
            target_pillar.setRelativePosition([np.random.uniform(-0.5, 0.5), np.random.uniform(-0.5, 0.5), .0])
            target_pillar.setColor([0,1,0])
        return C
        

### Define Komo problem

In [8]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))
C = create_n_boxes(C = C, N = 3, position = 'fixed')
goal = create_construction_site(C, position = 'fixed')
pillar_goal = create_pillar_target_postion(C, position = 'fixed')
C.view()

0

In [9]:
def gripper_to_block(C, bot, block = 'box1', prints = False):
    qHome = C.getJointState()

    komo = ry.KOMO(C, 1, 1, 0, False)

    komo.addObjective(
        times=[], 
        feature=ry.FS.jointState, 
        frames=[],
        type=ry.OT.sos, 
        scale=[1e-1], 
        target=qHome
    )
    komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
    komo.addObjective([], ry.FS.positionDiff, ['l_gripper', block], ry.OT.eq, [1e1])
    komo.addObjective([], ry.FS.scalarProductXZ, ['l_gripper', block], ry.OT.eq, [1e1], [0])
    komo.addObjective([], ry.FS.scalarProductXY, ['l_gripper', block], ry.OT.eq, [1e1], [1])

    komo.addObjective([], ry.FS.distance, ['l_palm', block], ry.OT.ineq, [1e1])
    komo.addObjective([1], ry.FS.vectorZ, ['l_gripper'], ry.OT.eq, [1e1], [0, 0 , 1])
    # no table collision
    komo.addObjective([], ry.FS.distance, ['l_palm', 'table'], ry.OT.ineq, [1e1], [-0.001])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()

    bot.moveTo(q[-1], 5)
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .5)

In [10]:
def move_gripper_to(C, bot, add_frame_obj_str = 'target', carry_item = 'box1', orientation = 'vertical', prints=False):
    komo = ry.KOMO(C, 1, 1, 1, True)
    komo.addControlObjective([], 0, 0.1e1)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', add_frame_obj_str], ry.OT.eq, [1e1])
    
    if orientation == 'vertical':
        komo.addObjective([1], ry.FS.vectorZ, ['l_gripper'], ry.OT.eq, [1e1], [0, 0 , 1])    
        komo.addObjective([1], ry.FS.vectorX, ['l_gripper'], ry.OT.eq, [1e1], [0, 1, 0])     

    if orientation == 'horizontal':
        komo.addObjective([1], ry.FS.vectorZ, ['l_gripper'], ry.OT.eq, [1e1], [1, 0, 0])
        komo.addObjective([1], ry.FS.vectorX, ['l_gripper'], ry.OT.eq, [1e1], [0, 1, 0])     

    # no table collision
    #komo.addObjective([], ry.FS.distance, ['l_palm', 'table'], ry.OT.ineq, [1e1])
    komo.addObjective([], ry.FS.negDistance, ['l_palm', 'table'], ry.OT.ineq, [1e1], [-0.175])
    
    # avoid palm collision with box
    komo.addObjective([], ry.FS.distance, ['l_gripper', carry_item], ry.OT.ineq, [1e1])
    

    # avoid box collision during transport of box
    if carry_item == 'box1':
        komo.addObjective([], ry.FS.distance, [carry_item, 'box2'], ry.OT.ineq, [1e1] )
    
    if carry_item == 'box1':
        komo.addObjective([], ry.FS.distance, [carry_item, 'box1'], ry.OT.ineq, [1e1])
    
    if carry_item == 'box3':
        komo.addObjective([], ry.FS.distance, [carry_item, 'box1'], ry.OT.ineq, [1e1])
        komo.addObjective([], ry.FS.distance, [carry_item, 'box2'], ry.OT.ineq, [1e1])

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1], 2)
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)

In [11]:
def retract_gripper(C, bot, distance = 0.25, prints = False):
    gripper_pos = C.getFrame("l_gripper").getPosition()
    retract_to_pos = gripper_pos + [0.0, 0.0, distance]
    C.addFrame('retract_to_pos').setShape(ry.ST.marker, [.1, .1, .1]).setPosition(retract_to_pos).setColor([1, .5, 1])

    komo = ry.KOMO(C, 1, 1, 1, True)
    komo.addControlObjective([], 0, 0.1e1)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
    komo.addObjective([1], ry.FS.positionDiff, ['l_gripper', 'retract_to_pos'], ry.OT.eq, [1e1])

    komo.addObjective([1], ry.FS.vectorZ, ['l_gripper'], ry.OT.eq, [1e1], [0, 0 , 1])    
    komo.addObjective([1], ry.FS.vectorX, ['l_gripper'], ry.OT.eq, [1e1], [0, 1, 0])     

    ret = ry.NLP_Solver(komo.nlp(), verbose=4) .solve()
    if prints == True:
        print(ret)
    
    q = komo.getPath()
    
    bot.moveTo(q[-1], 2)
    while bot.getTimeToEnd() > 0:
        bot.sync(C, .1)





In [12]:
bot = ry.BotOp(C, useRealRobot = False)

## First block

In [13]:
gripper_open(C, bot)
gripper_to_block(C, bot, block='box1')
gripper_close_grasp(bot, target_obj = 'box1', force = 5.0,  widht = 0.045)
move_gripper_to(C, bot, add_frame_obj_str = 'pillar_target_pos1', carry_item = 'box1', orientation = 'vertical')
gripper_open(C, bot)
retract_gripper(C, bot, distance = 0.15)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):79.6575 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    54.4448  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):      27.91  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):       12.4  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    5.44637  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1.73578  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):   0.176322  ACCEPT
--newton-- it:   7  |Delta|:   0.080895  alpha:          1  evals:   8  f(y):  0.0426954  ACCEPT
--newton-- it:   8  |Delta|:  0.0314055  alpha:          1  evals:   9  f(y):  0.0383359  ACCEPT
--newton-- it:   9  |Delta|:  0.0190444  alpha:          1  evals:  10  f(y

### Second Block

In [14]:
gripper_open(C, bot)
gripper_to_block(C, bot, block='box2')
gripper_close_grasp(bot, target_obj = 'box2', force = 5.0,  widht = 0.045)
move_gripper_to(C, bot, add_frame_obj_str = 'pillar_target_pos2', carry_item = 'box2', orientation = 'vertical')
gripper_open(C, bot)
retract_gripper(C, bot, distance = 0.15)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):74.5548 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    50.2179  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    24.9065  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    10.8209  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    4.49992  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1.23759  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):  0.0709132  ACCEPT
--newton-- it:   7  |Delta|:  0.0470003  alpha:          1  evals:   8  f(y):   0.040753  ACCEPT
--newton-- it:   8  |Delta|:  0.0211693  alpha:          1  evals:   9  f(y):  0.0380032  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0 

### 3rd Block

In [15]:
box_1 = C.getFrame('box1')
box_2 = C.getFrame('box2')
box_pos1 = box_1.getPosition()
box_pos2 = box_2.getPosition()
offset = 0.025
middle_between_towers = np.mean([box_pos1, box_pos2], 0) + [0, 0, offset]
C.addFrame('block3_target').setShape(ry.ST.marker, [.1, .1, .1]).setPosition(middle_between_towers).setColor([1, .5, 1])
C.view()

0

In [16]:
gripper_to_block(C, bot, block='box3')
gripper_close_grasp(bot, target_obj = 'box3', force = 5.0,  widht = 0.045)
move_gripper_to(C, bot, add_frame_obj_str = 'block3_target', carry_item = 'box3', orientation = 'horizontal')
gripper_open(C, bot)
bot.home(C)

====nlp==== method:AugmentedLagrangian bounded: yes
==nlp== it:0 evals:0 mu:1 nu:1 muLB:0.1
----newton---- initial point f(x):77.9892 alpha:1 beta:1
--newton-- it:   1  |Delta|:        0.2  alpha:          1  evals:   2  f(y):    52.9868  ACCEPT
--newton-- it:   2  |Delta|:        0.2  alpha:          1  evals:   3  f(y):    26.7018  ACCEPT
--newton-- it:   3  |Delta|:        0.2  alpha:          1  evals:   4  f(y):    11.8603  ACCEPT
--newton-- it:   4  |Delta|:        0.2  alpha:          1  evals:   5  f(y):    5.15854  ACCEPT
--newton-- it:   5  |Delta|:        0.2  alpha:          1  evals:   6  f(y):    1.58634  ACCEPT
--newton-- it:   6  |Delta|:        0.2  alpha:          1  evals:   7  f(y):   0.139845  ACCEPT
--newton-- it:   7  |Delta|:  0.0673023  alpha:          1  evals:   8  f(y):  0.0471918  ACCEPT
--newton-- it:   8  |Delta|:  0.0224884  alpha:          1  evals:   9  f(y):  0.0436279  ACCEPT
--newton-- stopping: 'absMax(Delta)<options.stopTolerance'
==nlp== it:   0 

### Clear config

In [17]:
C.clear()